In [120]:
from pymongo import MongoClient
from pprint import pprint
from bs4 import BeautifulSoup as bs
import requests
from pprint import pprint
import re

### Часть с НН

### Задание 1 

In [139]:
client = MongoClient('localhost', 27017) # Подключаемся к клиенту
db = client['vacancy_db'] # Создаем БД
hh = db.hh # Создаем объект, который создаст коллекцию hh в БД vacancy_db

main_link = 'https://hh.ru/search/vacancy'

header = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0'}

response = requests.get(main_link, headers=header).text
soup = bs(response, 'html.parser')

list_count = soup.find('div', {'data-qa': 'pager-block'})

q = int(str(list_count)[213])

for w in range(q):
    params = {'area': '1',
          'text': 'Data scientist',
            'st': 'searchVacancy',
          'fromSearch': True,
            'from': 'suggest_post',
         'page':w}
    response = requests.get(main_link, headers=header, params=params).text
    soup = bs(response, 'html.parser')
    jobs = soup.find_all('div', {'class': 'vacancy-serp-item__row vacancy-serp-item__row_header'})
    jobs_link = soup.find_all('a', {'class': 'bloko-link HH-LinkModifier'})

    for l in range(len(jobs_link)):

        a = re.sub('[0-9]', '', jobs[l].getText()).replace('-', '') # Наим. вакансии с валютой
              
        d = a.replace('руб.', '').replace(' ', '').replace('от', '')
    
        b = re.sub('[\D]', '', jobs[l].getText())   # Зарплата
        #print(b)
        if len(b) == 0:
            e = None
            e1 = None
        elif a.find('от') == -1:
            #e = b
            if a.find('до') == -1:
                if len(b)%2 == 0:
                    
                    e = int(b[:int(len(b)/2)])
                    e1 = int(b[int(len(b)/2):])
                else:
                    
                    e = int(b[:(int((len(b)-1)/2))])
                    e1 = int(b[(int((len(b)-1)/2)):])
            else:
                e = None
                e1 = int(b)
        else:
            e = int(b)
            e1 = None
            
        a = re.sub('[0-9]', '', jobs[l].getText()).replace('-', '').replace('до','') # Наим. вакансии с валютой
        if a.find('руб.') == -1:
            c = 'None'
        else:
            c = 'руб.'
           
        hh.insert_one({"Наименование вакансии": d,
                "зарплата min": e,
                "зарплата max": e1,
                "валюта": c,
                "cсылка на вакансию": jobs_link[l]['href'],
                "источник": 'hh.ru',
                "ID вакансии": re.sub('[\D]', '', jobs_link[l]['href'])[:-2]})

In [140]:
a = []
for i in hh.find():
    pprint(i)
    a.append(i.get('ID вакансии'))

{'ID вакансии': '37635578',
 '_id': ObjectId('5efdb8bc33f4d8960dcb0743'),
 'cсылка на вакансию': 'https://hh.ru/vacancy/37635578?query=Data%20scientist',
 'Наименование вакансии': 'DataScientist\xa0\xa0',
 'валюта': 'руб.',
 'зарплата max': 180000,
 'зарплата min': 120000,
 'источник': 'hh.ru'}
{'ID вакансии': '37521387',
 '_id': ObjectId('5efdb8bc33f4d8960dcb0744'),
 'cсылка на вакансию': 'https://hh.ru/vacancy/37521387?query=Data%20scientist',
 'Наименование вакансии': 'PythonDataScientist\xa0',
 'валюта': 'руб.',
 'зарплата max': None,
 'зарплата min': 150000,
 'источник': 'hh.ru'}
{'ID вакансии': '37569615',
 '_id': ObjectId('5efdb8bc33f4d8960dcb0745'),
 'cсылка на вакансию': 'https://hh.ru/vacancy/37569615?query=Data%20scientist',
 'Наименование вакансии': 'DataScientist\xa0',
 'валюта': 'руб.',
 'зарплата max': None,
 'зарплата min': 130000,
 'источник': 'hh.ru'}
{'ID вакансии': '37559909',
 '_id': ObjectId('5efdb8bc33f4d8960dcb0746'),
 'cсылка на вакансию': 'https://hh.ru/vacanc

In [142]:
#hh.delete_many({})

### Задание 2

In [143]:
a = int(input('Введите значение зарплаты: '))
b = a
for i in hh.find({'зарплата max':{'$gt':b}, 'зарплата min':{'$gt':b}}):
    pprint(i)

Введите значение зарплаты: 120000
{'ID вакансии': '37506599',
 '_id': ObjectId('5efdb8be33f4d8960dcb07ad'),
 'cсылка на вакансию': 'https://hh.ru/vacancy/37506599?query=Data%20scientist',
 'Наименование вакансии': 'Директорпоинвестициям(Familyoffice)\xa0\xa0\xa0',
 'валюта': 'руб.',
 'зарплата max': 1000000,
 'зарплата min': 500000,
 'источник': 'hh.ru'}
{'ID вакансии': '37618147',
 '_id': ObjectId('5efdb8be33f4d8960dcb07b0'),
 'cсылка на вакансию': 'https://hh.ru/vacancy/37618147?query=Data%20scientist',
 'Наименование вакансии': 'SeniorPythonSystemsEngineer\xa0\xa0',
 'валюта': 'руб.',
 'зарплата max': 280000,
 'зарплата min': 200000,
 'источник': 'hh.ru'}


### Задание 3

In [144]:
v = {} # словарь с ключем 'ID вакансии' и значением ключа '_id'
for i in hh.find():
    c = i.get('ID вакансии')
    d = i.get('_id')
    v.update({c:d})
print(f'Перед обновлением в коллекции было {len(v)} вакансии')

Перед обновлением в коллекции было 162 вакансии


In [145]:
client = MongoClient('localhost', 27017) # Подключаемся к клиенту
db = client['vacancy_db'] # Создаем БД
hh = db.hh # Создаем объект, который создаст коллекцию hh в БД vacancy_db

main_link = 'https://hh.ru/search/vacancy'

header = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0'}

response = requests.get(main_link, headers=header).text
soup = bs(response, 'html.parser')

list_count = soup.find('div', {'data-qa': 'pager-block'})

q = int(str(list_count)[213])


def add_new_vacancy(v):
    
    if (re.sub('[\D]', '', jobs_link[l]['href'])[:-2]) not in v.keys():
        hh.insert_one({"Наименование вакансии": d,
                        "зарплата min": e,
                        "зарплата max": e1,
                        "валюта": c,
                        "cсылка на вакансию": jobs_link[l]['href'],
                        "источник": 'hh.ru',
                        "ID вакансии": re.sub('[\D]', '', jobs_link[l]['href'])[:-2]})

for w in range(q):
    params = {'area': '1',
          'text': 'Data scientist',
            'st': 'searchVacancy',
          'fromSearch': True,
            'from': 'suggest_post',
         'page':w}
    response = requests.get(main_link, headers=header, params=params).text
    soup = bs(response, 'html.parser')
    jobs = soup.find_all('div', {'class': 'vacancy-serp-item__row vacancy-serp-item__row_header'})
    jobs_link = soup.find_all('a', {'class': 'bloko-link HH-LinkModifier'})

    for l in range(len(jobs_link)):
        
        a = re.sub('[0-9]', '', jobs[l].getText()).replace('-', '') # Наим. вакансии с валютой
              
        d = a.replace('руб.', '').replace(' ', '').replace('от', '')
    
        b = re.sub('[\D]', '', jobs[l].getText())   # Зарплата
        #print(b)
        if len(b) == 0:
            e = None
            e1 = None
        elif a.find('от') == -1:
            #e = b
            if a.find('до') == -1:
                if len(b)%2 == 0:
                    
                    e = int(b[:int(len(b)/2)])
                    e1 = int(b[int(len(b)/2):])
                else:
                    
                    e = int(b[:(int((len(b)-1)/2))])
                    e1 = int(b[(int((len(b)-1)/2)):])
            else:
                e = None
                e1 = int(b)
        else:
            e = int(b)
            e1 = None
            
        a = re.sub('[0-9]', '', jobs[l].getText()).replace('-', '').replace('до','') # Наим. вакансии с валютой
        if a.find('руб.') == -1:
            c = 'None'
        else:
            c = 'руб.'
        add_new_vacancy(v)
        

In [146]:
j = []
for i in hh.find():    
    j.append(i.get('ID вакансии'))
print(f'В результате обновления добавлено {len(j) - len(v)} новых вакансий')

В результате обновления добавлено 47 новых вакансий


### Часть с SJ

### Задание 1

In [149]:
client = MongoClient('localhost', 27017) # Подключаемся к клиенту
db = client['vacancy_db'] # Создаем БД
sj = db.sj # Создаем объект, который создаст коллекцию sj в БД vacancy_db
main_link = 'https://www.superjob.ru/vacancy/search/'

header = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0'}

params = {'keywords': 'DataScientist'}
response = requests.get(main_link, headers=header, params=params).text
soup = bs(response, 'html.parser')

jobs_sj = soup.find_all('div', {'class': '_3mfro PlM3e _2JVkc _3LJqf'})

jobs_sj_salary = soup.find_all('span', {'class': '_3mfro _2Wp8I PlM3e _2JVkc _2VHxz'})

jobs_sj = soup.find_all('div', {'class': '_3mfro PlM3e _2JVkc _3LJqf'})

for i in range(len(jobs_sj)):
   
    #pprint(jobs_sj[i].getText()) # Наименование вакансии
    
    d = jobs_sj[i].getText()
    
    #pprint(jobs_sj_salary[i].getText()) # Вознаграждение
    
    e = jobs_sj_salary[i].getText()
    
    
    b = re.sub('[\D]', '', e)   # Зарплата
        #print(b)
    if len(b) == 0:
        u = None
        u1 = None
    elif e.find('от') == -1:
        #e = b
        if e.find('до') == -1:
            if len(b)%2 == 0:
                    
                u = int(b[:int(len(b)/2)])
                u1 = int(b[int(len(b)/2):])
            else:
                    
                u = int(b[:(int((len(b)-1)/2))])
                u1 = int(b[(int((len(b)-1)/2)):])
        else:
            u = None
            u1 = int(b)
    else:
        u = int(b)
        u = None
    
    
    r = str(jobs_sj[i]).split('"')
    
    #print('https://www.superjob.ru' + r[5]) # Ссылка на вакансию
    
    t = 'https://www.superjob.ru' + r[5]
    
    if e.find('руб.') == -1:
        y = 'None'
    else:
        y = 'руб.'
         
    sj.insert_one({"Наименование вакансии": d,
            "зарплата min": u,
            "зарплата max": u1,
            "валюта": y,
            "cсылка на вакансию": t,
            "источник": 'superjob.ru',
            "ID вакансии": re.sub('[\D]', '', r[5])})

In [150]:
p = []
for i in sj.find():
    pprint(i)    
    p.append(i.get('ID вакансии'))

{'ID вакансии': '33947478',
 '_id': ObjectId('5efdbe4333f4d8960dcb081c'),
 'cсылка на вакансию': 'https://www.superjob.ru/vakansii/data-scientist-33947478.html',
 'Наименование вакансии': 'Data Scientist',
 'валюта': 'None',
 'зарплата max': None,
 'зарплата min': None,
 'источник': 'superjob.ru'}
{'ID вакансии': '33864238',
 '_id': ObjectId('5efdbe4333f4d8960dcb081d'),
 'cсылка на вакансию': 'https://www.superjob.ru/vakansii/genetik-analitik-dannyh-33864238.html',
 'Наименование вакансии': 'Генетик-аналитик данных / Data scientist',
 'валюта': 'None',
 'зарплата max': None,
 'зарплата min': None,
 'источник': 'superjob.ru'}
{'ID вакансии': '33964334',
 '_id': ObjectId('5efdbe4333f4d8960dcb081e'),
 'cсылка на вакансию': 'https://www.superjob.ru/vakansii/fullstek-razrabotchik-33964334.html',
 'Наименование вакансии': 'Фуллстек разработчик / Senior Software Developer',
 'валюта': 'None',
 'зарплата max': None,
 'зарплата min': None,
 'источник': 'superjob.ru'}
{'ID вакансии': '34064156',

### Задание 2

In [151]:
o = int(input('Введите значение зарплаты: '))
f = o
for i in sj.find({'зарплата max':{'$gt':f}, 'зарплата min':{'$gt':f}}):
    pprint(i)

Введите значение зарплаты: 100000


### Задание 3

In [152]:
h = {}
for i in sj.find():
    c = i.get('ID вакансии')
    d = i.get('_id')
    h.update({c:d})
print(f'Перед обновлением в коллекции было {len(h)} вакансии')

Перед обновлением в коллекции было 5 вакансии


In [157]:
client = MongoClient('localhost', 27017) # Подключаемся к клиенту
db = client['vacancy_db'] # Создаем БД
sj = db.sj # Создаем объект, который создаст коллекцию sj в БД vacancy_db
main_link = 'https://www.superjob.ru/vacancy/search/'

header = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0'}

params = {'keywords': 'DataScientist'}
response = requests.get(main_link, headers=header, params=params).text
soup = bs(response, 'html.parser')

def add_new_vacancy(v):
    
    if (re.sub('[\D]', '', r[5])) not in v.keys():
        sj.insert_one({"Наименование вакансии": d,
                "зарплата min": u,
                "зарплата max": u1,
                "валюта": y,
                "cсылка на вакансию": t,
                "источник": 'superjob.ru',
                "ID вакансии": re.sub('[\D]', '', r[5])})
        

jobs_sj = soup.find_all('div', {'class': '_3mfro PlM3e _2JVkc _3LJqf'})

jobs_sj_salary = soup.find_all('span', {'class': '_3mfro _2Wp8I PlM3e _2JVkc _2VHxz'})

jobs_sj = soup.find_all('div', {'class': '_3mfro PlM3e _2JVkc _3LJqf'})

for i in range(len(jobs_sj)):
   
    #pprint(jobs_sj[i].getText()) # Наименование вакансии
    
    d = jobs_sj[i].getText()
    
    #pprint(jobs_sj_salary[i].getText()) # Вознаграждение
    
    e = jobs_sj_salary[i].getText()
    
    
    b = re.sub('[\D]', '', e)   # Зарплата
    if len(b) == 0:
        u = None
        u1 = None
    elif e.find('от') == -1:
        if e.find('до') == -1:
            if len(b)%2 == 0:
                    
                u = int(b[:int(len(b)/2)])
                u1 = int(b[int(len(b)/2):])
            else:
                    
                u = int(b[:(int((len(b)-1)/2))])
                u1 = int(b[(int((len(b)-1)/2)):])
        else:
            u = None
            u1 = int(b)
    else:
        u = int(b)
        u = None
    
    
    r = str(jobs_sj[i]).split('"')
    
    t = 'https://www.superjob.ru' + r[5]
    
    if e.find('руб.') == -1:
        y = 'None'
    else:
        y = 'руб.'
    add_new_vacancy(h)

In [159]:
l = []
for i in sj.find():
    l.append(i.get('ID вакансии'))
print(f'В результате обновления добавлено {len(l) - len(h)} новых вакансий')

В результате обновления добавлено 0 новых вакансий
